# Lora 实战

## Step1 导入相关包

In [1]:
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, DataCollatorForSeq2Seq, TrainingArguments, Trainer

bin c:\Users\user\AppData\Local\Programs\Python\Python311\Lib\site-packages\bitsandbytes\libbitsandbytes_cuda121.dll


## Step2 加载数据集

In [2]:
ds = Dataset.load_from_disk("../data/alpaca_data_zh/")
ds

Dataset({
    features: ['output', 'input', 'instruction'],
    num_rows: 26858
})

In [3]:
ds[:3]

{'output': ['以下是保持健康的三个提示：\n\n1. 保持身体活动。每天做适当的身体运动，如散步、跑步或游泳，能促进心血管健康，增强肌肉力量，并有助于减少体重。\n\n2. 均衡饮食。每天食用新鲜的蔬菜、水果、全谷物和脂肪含量低的蛋白质食物，避免高糖、高脂肪和加工食品，以保持健康的饮食习惯。\n\n3. 睡眠充足。睡眠对人体健康至关重要，成年人每天应保证 7-8 小时的睡眠。良好的睡眠有助于减轻压力，促进身体恢复，并提高注意力和记忆力。',
  '4/16等于1/4是因为我们可以约分分子分母都除以他们的最大公约数4，得到（4÷4）/ (16÷4）=1/4。分数的约分是用分子和分母除以相同的非零整数，来表示分数的一个相同的值，这因为分数实际上表示了分子除以分母，所以即使两个数同时除以同一个非零整数，分数的值也不会改变。所以4/16 和1/4是两种不同的书写形式，但它们的值相等。',
  '朱利叶斯·凯撒，又称尤利乌斯·恺撒（Julius Caesar）是古罗马的政治家、军事家和作家。他于公元前44年3月15日被刺杀。 \n\n根据历史记载，当时罗马元老院里一些参议员联合起来策划了对恺撒的刺杀行动，因为他们担心恺撒的统治将给罗马共和制带来威胁。在公元前44年3月15日（又称“3月的艾达之日”），恺撒去参加元老院会议时，被一群参议员包围并被攻击致死。据记载，他身中23刀，其中一刀最终致命。'],
 'input': ['', '输入：4/16', ''],
 'instruction': ['保持健康的三个提示。', '解释为什么以下分数等同于1/4', '朱利叶斯·凯撒是如何死亡的？']}

In [ ]:
# print(len("以下是保持健康的三个提示：\n\n1. 保持身体活动。每天做适当的身体运动，如散步、跑步或游泳，能促进心血管健康，增强肌肉力量，并有助于减少体重。\n\n2. 均衡饮食。每天食用新鲜的蔬菜、水果、全谷物和脂肪含量低的蛋白质食物，避免高糖、高脂肪和加工食品，以保持健康的饮食习惯。\n\n3. 睡眠充足。睡眠对人体健康至关重要，成年人每天应保证 7-8 小时的睡眠。良好的睡眠有助于减轻压力，促进身体恢复，并提高注意力和记忆力。"))

## Step3 数据集预处理

In [5]:
model_path = r"C:\Users\user\.cache\huggingface\hub\models--yentinglin--Taiwan-LLM-7B-v2.1-chat\snapshots\be7d3a45160727c67088fda933c1413fa97138a2"
tokenizer = AutoTokenizer.from_pretrained(model_path)
# tokenizer = AutoTokenizer.from_pretrained(r"C:\Users\user\.cache\huggingface\hub\models--yentinglin--Taiwan-LLM-7B-v2.1-chat")
tokenizer

LlamaTokenizerFast(name_or_path='C:\Users\user\.cache\huggingface\hub\models--yentinglin--Taiwan-LLM-7B-v2.1-chat\snapshots\be7d3a45160727c67088fda933c1413fa97138a2', vocab_size=32000, model_max_length=4096, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<unk>'}, clean_up_tokenization_spaces=False),  added_tokens_decoder={
	0: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

In [6]:
tokenizer.padding_side = "right"  # 一定要设置padding_side为right，否则batch大于1时可能不收敛

In [7]:
tokenizer.pad_token_id = 2

In [8]:
def process_func(example):
    MAX_LENGTH = 384    # Llama分词器会将一个中文字切分为多个token，因此需要放开一些最大长度，保证数据的完整性
    input_ids, attention_mask, labels = [], [], []
    instruction = tokenizer("\n".join(["Human: " + example["instruction"], example["input"]]).strip() + "\n\nAssistant: ", add_special_tokens=False)
    response = tokenizer(example["output"], add_special_tokens=False)
    input_ids = instruction["input_ids"] + response["input_ids"] + [tokenizer.eos_token_id]
    attention_mask = instruction["attention_mask"] + response["attention_mask"] + [1]
    labels = [-100] * len(instruction["input_ids"]) + response["input_ids"] + [tokenizer.eos_token_id]
    if len(input_ids) > MAX_LENGTH:
        input_ids = input_ids[:MAX_LENGTH]
        attention_mask = attention_mask[:MAX_LENGTH]
        labels = labels[:MAX_LENGTH]
    return {
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "labels": labels
    }

In [9]:
tokenized_ds = ds.map(process_func, remove_columns=ds.column_names)
tokenized_ds

Map:   0%|          | 0/26858 [00:00<?, ? examples/s]

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 26858
})

In [10]:
print(tokenized_ds[0]["input_ids"])

[12968, 29901, 29871, 30982, 31695, 31863, 31577, 30210, 30457, 30502, 31302, 30858, 30267, 13, 13, 7900, 22137, 29901, 29871, 29871, 30651, 30557, 30392, 30982, 31695, 31863, 31577, 30210, 30457, 30502, 31302, 30858, 30383, 13, 13, 29896, 29889, 29871, 30982, 31695, 31687, 30988, 31704, 30846, 30267, 31951, 30408, 232, 132, 157, 236, 131, 133, 30948, 30210, 31687, 30988, 31894, 30846, 30214, 30847, 233, 152, 166, 233, 176, 168, 30330, 235, 186, 148, 233, 176, 168, 31391, 233, 187, 187, 233, 182, 182, 30214, 30815, 231, 194, 134, 31174, 30869, 235, 164, 131, 31624, 31863, 31577, 30214, 232, 165, 161, 232, 191, 189, 235, 133, 143, 235, 133, 140, 31074, 31180, 30214, 31666, 30417, 31931, 30909, 232, 138, 146, 31022, 30988, 30908, 30267, 13, 13, 29906, 29889, 29871, 232, 160, 138, 235, 164, 164, 236, 168, 177, 31855, 30267, 31951, 30408, 31855, 30406, 30374, 236, 181, 159, 30210, 235, 151, 175, 31854, 30330, 30716, 30801, 30330, 30753, 31112, 30834, 30503, 235, 135, 133, 235, 133, 173, 23

In [11]:
# tokenizer("abc " + tokenizer.eos_token)

In [12]:
tokenizer.decode(tokenized_ds[0]["input_ids"])

'Human: 保持健康的三个提示。\n\nAssistant:  以下是保持健康的三个提示：\n\n1. 保持身体活动。每天做适当的身体运动，如散步、跑步或游泳，能促进心血管健康，增强肌肉力量，并有助于减少体重。\n\n2. 均衡饮食。每天食用新鲜的蔬菜、水果、全谷物和脂肪含量低的蛋白质食物，避免高糖、高脂肪和加工食品，以保持健康的饮食习惯。\n\n3. 睡眠充足。睡眠对人体健康至关重要，成年人每天应保证 7-8 小时的睡眠。良好的睡眠有助于减轻压力，促进身体恢复，并提高注意力和记忆力。</s>'

In [13]:
# tokenizer("呀", add_special_tokens=False) # Llama分词器会将一个中文字切分为多个token，因此需要放开一些最大长度，保证数据的完整性

In [14]:
tokenizer.decode(list(filter(lambda x: x != -100, tokenized_ds[1]["labels"])))

'4/16等于1/4是因为我们可以约分分子分母都除以他们的最大公约数4，得到（4÷4）/ (16÷4）=1/4。分数的约分是用分子和分母除以相同的非零整数，来表示分数的一个相同的值，这因为分数实际上表示了分子除以分母，所以即使两个数同时除以同一个非零整数，分数的值也不会改变。所以4/16 和1/4是两种不同的书写形式，但它们的值相等。</s>'

## Step4 创建模型

In [17]:
import torch
model_path = r"C:\Users\user\.cache\huggingface\hub\models--yentinglin--Taiwan-LLM-7B-v2.1-chat\snapshots\be7d3a45160727c67088fda933c1413fa97138a2"
model = AutoModelForCausalLM.from_pretrained(model_path, low_cpu_mem_usage=True, torch_dtype=torch.half, device_map="auto")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [18]:
model.dtype

torch.float16

## Lora

### PEFT Step1 配置文件

In [19]:
from peft import LoraConfig, TaskType, get_peft_model

config = LoraConfig(task_type=TaskType.CAUSAL_LM,)
config

LoraConfig(peft_type=<PeftType.LORA: 'LORA'>, auto_mapping=None, base_model_name_or_path=None, revision=None, task_type=<TaskType.CAUSAL_LM: 'CAUSAL_LM'>, inference_mode=False, r=8, target_modules=None, lora_alpha=8, lora_dropout=0.0, fan_in_fan_out=False, bias='none', modules_to_save=None, init_lora_weights=True, layers_to_transform=None, layers_pattern=None)

### PEFT Step2 创建模型

In [20]:
model = get_peft_model(model, config)

CUDA extension not installed.
CUDA extension not installed.


In [21]:
config

LoraConfig(peft_type=<PeftType.LORA: 'LORA'>, auto_mapping=None, base_model_name_or_path='C:\\Users\\user\\.cache\\huggingface\\hub\\models--yentinglin--Taiwan-LLM-7B-v2.1-chat\\snapshots\\be7d3a45160727c67088fda933c1413fa97138a2', revision=None, task_type=<TaskType.CAUSAL_LM: 'CAUSAL_LM'>, inference_mode=False, r=8, target_modules=['q_proj', 'v_proj'], lora_alpha=8, lora_dropout=0.0, fan_in_fan_out=False, bias='none', modules_to_save=None, init_lora_weights=True, layers_to_transform=None, layers_pattern=None)

In [22]:
# model.enable_input_require_grads() # 开启梯度检查点时，要执行该方法

In [23]:
# model = model.half()  # 当整个模型都是半精度时，需要将adam_epsilon调大
# torch.tensor(1e-8).half() 

In [24]:
model.print_trainable_parameters()

trainable params: 4,194,304 || all params: 6,742,609,920 || trainable%: 0.06220594176090199


In [25]:
from torch.utils.data import DataLoader

In [26]:
dl = DataLoader(tokenized_ds, batch_size=2, collate_fn=DataCollatorForSeq2Seq(tokenizer=tokenizer, padding=True))

In [27]:
ipt = next(enumerate(dl))[1]

In [28]:
ipt

{'input_ids': tensor([[12968, 29901, 29871, 30982, 31695, 31863, 31577, 30210, 30457, 30502,
         31302, 30858, 30267,    13,    13,  7900, 22137, 29901, 29871, 29871,
         30651, 30557, 30392, 30982, 31695, 31863, 31577, 30210, 30457, 30502,
         31302, 30858, 30383,    13,    13, 29896, 29889, 29871, 30982, 31695,
         31687, 30988, 31704, 30846, 30267, 31951, 30408,   232,   132,   157,
           236,   131,   133, 30948, 30210, 31687, 30988, 31894, 30846, 30214,
         30847,   233,   152,   166,   233,   176,   168, 30330,   235,   186,
           148,   233,   176,   168, 31391,   233,   187,   187,   233,   182,
           182, 30214, 30815,   231,   194,   134, 31174, 30869,   235,   164,
           131, 31624, 31863, 31577, 30214,   232,   165,   161,   232,   191,
           189,   235,   133,   143,   235,   133,   140, 31074, 31180, 30214,
         31666, 30417, 31931, 30909,   232,   138,   146, 31022, 30988, 30908,
         30267,    13,    13, 29906, 2

In [29]:
model(**ipt.to("cuda"))

CausalLMOutputWithPast(loss=tensor(0.9918, device='cuda:0', grad_fn=<NllLossBackward0>), logits=tensor([[[ -8.2734, -10.0547,   2.9570,  ...,  -2.0938,  -3.6543, -10.4844],
         [ -9.7500,  -8.3359,   6.3711,  ...,  -2.1094,  -5.5664,  -4.3633],
         [  0.7114,   1.2168,   7.8164,  ...,  10.6719,   5.9180,   4.8867],
         ...,
         [  0.8364,   1.6221,  13.8125,  ...,   6.9414,   1.3389,   7.6211],
         [  2.8691,   3.7559,  20.5469,  ...,   9.1953,   5.0703,  10.0156],
         [  4.4492,  15.5781,  23.3438,  ...,   4.6836,   0.2917,   2.2852]],

        [[ -8.2734, -10.0547,   2.9570,  ...,  -2.0938,  -3.6543, -10.4844],
         [ -9.7500,  -8.3359,   6.3711,  ...,  -2.1094,  -5.5664,  -4.3633],
         [  0.7114,   1.2168,   7.8164,  ...,  10.6719,   5.9180,   4.8867],
         ...,
         [  0.2406,  15.7812,  16.7344,  ...,   1.1504,   0.3423,   0.4236],
         [  0.4446,  16.2188,  17.1250,  ...,   1.4307,   0.4456,   0.5493],
         [  0.5229,  16.359

## Step5 配置训练参数

In [30]:
args = TrainingArguments(
    output_dir="./chatbot",
    per_device_train_batch_size=2,
    gradient_accumulation_steps=8,
    logging_steps=10,
    num_train_epochs=1,
)

## Step6 创建训练器

In [31]:
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_ds.select(range(6000)),
    data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer, padding=True),
)

## Step7 模型训练

In [32]:
trainer.train()

  0%|          | 0/375 [00:00<?, ?it/s]

{'loss': 1.088, 'grad_norm': 0.13816528022289276, 'learning_rate': 4.866666666666667e-05, 'epoch': 0.03}
{'loss': 1.0847, 'grad_norm': 0.21005527675151825, 'learning_rate': 4.7333333333333336e-05, 'epoch': 0.05}
{'loss': 1.0301, 'grad_norm': 0.1916995644569397, 'learning_rate': 4.600000000000001e-05, 'epoch': 0.08}
{'loss': 0.9865, 'grad_norm': 0.26269447803497314, 'learning_rate': 4.466666666666667e-05, 'epoch': 0.11}
{'loss': 0.9117, 'grad_norm': 0.22034862637519836, 'learning_rate': 4.3333333333333334e-05, 'epoch': 0.13}
{'loss': 0.9034, 'grad_norm': 0.39021778106689453, 'learning_rate': 4.2e-05, 'epoch': 0.16}
{'loss': 0.9482, 'grad_norm': 0.3830936849117279, 'learning_rate': 4.066666666666667e-05, 'epoch': 0.19}
{'loss': 0.9269, 'grad_norm': 0.40290969610214233, 'learning_rate': 3.933333333333333e-05, 'epoch': 0.21}
{'loss': 0.8403, 'grad_norm': 0.4012512266635895, 'learning_rate': 3.8e-05, 'epoch': 0.24}
{'loss': 0.8789, 'grad_norm': 0.34266719222068787, 'learning_rate': 3.666666

KeyboardInterrupt: 

## Step8 模型推理

In [ ]:
model.eval()
ipt = tokenizer("Human: {}\n{}".format("你好", "").strip() + "\n\nAssistant: ", return_tensors="pt").to(model.device)
tokenizer.decode(model.generate(**ipt, max_length=512, do_sample=True, eos_token_id=tokenizer.eos_token_id)[0], skip_special_tokens=True)

 